### Evaluating CPM with CEBaB
Our Causal Proxy Model (CPM) is for providing concept-based explanation for a blackbox model. We use newly developed CEBaB benchmark for comparing CPM with other concept-based explanation methods. This notebook evaluates CPM with CEBaB benchmark under different settings.

More importantly, we introduce new baselines for CPM as well. Formally, we evaluate the blackbox model with interchange intervention evaluation (which will be introduced in details below).

In this notebook, we can evaluate the following models:
- CPM: `BERT-base-uncased`
- CPM: `RoBERTa-base`
- CPM: `GPT2`
- CPM: `LSTM+GloVe`
- CPM: `Control`

and we can evaluate with the following conditions:
- 2-class
- 3-class
- 5-class

and this script also support evaluates experiments with different control conditions:
- `control=True`
- `control=False`
- `control="pretrain"`
- `control="random"`
- `control="finetune"`
- `control="fewshots"` (supports BERT only)
- `control="fewshots-augment"` (supports BERT only)
- `control="layer"` (supports BERT only)

#### Imports and Libs

In [1]:
from libs import *
from modelings.modelings_bert import *
from modelings.modelings_roberta import *
from modelings.modelings_gpt2 import *
from modelings.modelings_lstm import *
"""
For evaluate, we use a single random seed, as
the models are trained with 5 different seeds
already.
"""
_ = random.seed(123)
_ = np.random.seed(123)
_ = torch.manual_seed(123)

#### Main evaluate script

In [29]:
"""
The following blocks will run CEBaB benchmark in
all the combinations of the following conditions.
"""
grid = {
    "seed": [42],
    "h_dim": [192], # 1, 4, 16, 32, 64, 128, 192
    "class_num": [5],
    "control": ["fewshots-augment-large-epochs"],
    # True, False, pretrain, random, finetune, 
    # fewshots fewshots-augment fewshots-layer
    # fewshots-hdim
    # fewshots-augment-balance
    # fewshots-augment-large-epochs
    "beta" : [1.0],
    "gemma" : [3.0],
    "cls_dropout" : [0.1],
    "enc_dropout" : [0.1],
    "model_arch" : ["bert-base-uncased"],
    "true_cfc" : [5, 50, 200, 600, 1200, 1755], 
    # only used for fewshots evaluations.
    # 5, 50, 200, 600, 1200, 1755
    "interchange_layer" : [12]
}

keys, values = zip(*grid.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

device = 'cuda:0'
batch_size = 32

In [31]:
results = {}
for i in range(len(permutations_dicts)):
    
    seed=permutations_dicts[i]["seed"]
    class_num=permutations_dicts[i]["class_num"]
    beta=permutations_dicts[i]["beta"]
    gemma=permutations_dicts[i]["gemma"]
    h_dim=permutations_dicts[i]["h_dim"]
    dataset_type = f'{class_num}-way'
    correction_epsilon=None
    cls_dropout=permutations_dicts[i]["cls_dropout"]
    enc_dropout=permutations_dicts[i]["enc_dropout"]
    control=permutations_dicts[i]["control"]
    model_arch=permutations_dicts[i]["model_arch"]
    true_cfc=permutations_dicts[i]["true_cfc"]
    interchange_layer=permutations_dicts[i]["interchange_layer"]
    
    if model_arch == "bert-base-uncased":
        model_path = "BERT-results" if control == False \
            else "BERT-control-results" if control == True \
            else f"BERT-{control}-results"
        model_module = BERTForCEBaB
        explainer_module = CausalProxyModelForBERT
    elif model_arch == "roberta-base":
        model_path = "RoBERTa-results" if control == False \
            else "RoBERTa-control-results" if control == True \
            else f"RoBERTa-{control}-results" 
        model_module = RoBERTaForCEBaB
        explainer_module = CausalProxyModelForRoBERTa
    elif model_arch == "gpt2":
        model_path = "gpt2-results" if control == False \
            else "gpt2-control-results" if control == True \
            else f"gpt2-{control}-results"
        model_module = GPT2ForCEBaB
        explainer_module = CausalProxyModelForGPT2
    elif model_arch == "lstm":
        model_path = "lstm-results" if control == False \
            else "lstm-control-results" if control == True \
            else f"lstm-{control}-results"
        model_module = LSTMForCEBaB
        explainer_module = CausalProxyModelForLSTM
        
    grid_conditions=(
        ("seed", seed),
        ("class_num", class_num),
        ("beta", beta),
        ("gemma", gemma),
        ("h_dim", h_dim),
        ("dataset_type", dataset_type),
        ("correction_epsilon", correction_epsilon),
        ("cls_dropout", cls_dropout),
        ("enc_dropout", enc_dropout),
        ("control", control),
        ("model_arch", model_arch),
        ("true_cfc", true_cfc),
        ("interchange_layer", interchange_layer)
    )
    print("Running for this setting: ", grid_conditions)

    blackbox_model_path = f'CEBaB/{model_arch}.CEBaB.sa.{class_num}-class.exclusive.seed_{seed}'
    if control == "finetune": # not for other control cases, e.g., random or pretrain
        cpm_model_path = blackbox_model_path
    else:
        split_name = "train"
        cpm_model_path = f'../proxy_training_results/{model_path}/'\
                           f'cebab.train.{split_name}.alpha.1.0'\
                           f'.beta.{beta}.gemma.{gemma}.dim.{h_dim}.hightype.'\
                           f'{model_arch}.Proxy.'\
                           f'CEBaB.sa.{class_num}-class.exclusive.'\
                           f'mode.align.cls.dropout.{cls_dropout}.enc.dropout.{enc_dropout}'
        if control == "fewshots" or control == "fewshots-hdim":
            cpm_model_path += f'.true.cfc.{true_cfc}.aug.cfc.False'
        elif control == "fewshots-augment" or control == "fewshots-augment-balance" or control == "fewshots-augment-large-epochs":
            cpm_model_path += f'.true.cfc.{true_cfc}.aug.cfc.True'
        elif control == "fewshots-layer":
            cpm_model_path += f'.true.cfc.{true_cfc}.aug.cfc.False.int.layer.{interchange_layer}'
        cpm_model_path += f'.seed_{seed}'
    # load data from HF
    cebab = datasets.load_dataset(
        'CEBaB/CEBaB', use_auth_token=True,
        cache_dir="../train_cache/"
    )
    train, dev, test = preprocess_hf_dataset(
        cebab, one_example_per_world=True, 
        verbose=1, dataset_type=dataset_type
    )

    tf_model = model_module(
        blackbox_model_path, 
        device=device, 
        batch_size=batch_size
    )
    explanator = explainer_module(
        blackbox_model_path,
        cpm_model_path, 
        device=device, 
        batch_size=batch_size,
        intervention_h_dim=h_dim,
    )

    train_dataset = train.copy()
    dev_dataset = test.copy()

    result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
    CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
    ACaCE_per_aspect, performance_report = cebab_pipeline(
        tf_model, explanator, 
        train_dataset, dev_dataset, 
        dataset_type=dataset_type,
        correction_epsilon=correction_epsilon,
    )
    
    results[grid_conditions] = (
        result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
        CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
        ACaCE_per_aspect, performance_report
    )

Running for this setting:  (('seed', 42), ('class_num', 5), ('beta', 1.0), ('gemma', 3.0), ('h_dim', 192), ('dataset_type', '5-way'), ('correction_epsilon', None), ('cls_dropout', 0.1), ('enc_dropout', 0.1), ('control', 'fewshots-augment-large-epochs'), ('model_arch', 'bert-base-uncased'), ('true_cfc', 5), ('interchange_layer', 12))


Using custom data configuration CEBaB--CEBaB-0e2f7ed67c9d7e55
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-0e2f7ed67c9d7e55/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/4 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train dataset.
intervention_h_dim=192


100%|██████████| 124/124 [00:24<00:00,  5.03it/s]


...

Running for this setting:  (('seed', 42), ('class_num', 5), ('beta', 1.0), ('gemma', 3.0), ('h_dim', 192), ('dataset_type', '5-way'), ('correction_epsilon', None), ('cls_dropout', 0.1), ('enc_dropout', 0.1), ('control', 'fewshots-augment-large-epochs'), ('model_arch', 'bert-base-uncased'), ('true_cfc', 1755), ('interchange_layer', 12))


Using custom data configuration CEBaB--CEBaB-0e2f7ed67c9d7e55
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-0e2f7ed67c9d7e55/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/4 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train dataset.
intervention_h_dim=192


100%|██████████| 124/124 [00:24<00:00,  4.96it/s]


#### Show your results

In [32]:
important_keys = [
    "seed", "h_dim", "class_num", 
    "control", "beta", "gemma", 
    "cls_dropout", "enc_dropout", 
    "model_arch", "true_cfc", 
    "interchange_layer"
]
values = []
for k, v in results.items():
    _values = []
    for ik in important_keys:
        _values.append(dict(k)[ik])
    _values.append(v[2]["ICaCE-L2"].iloc[0])
    _values.append(v[2]["ICaCE-cosine"].iloc[0])
    _values.append(v[2]["ICaCE-normdiff"].iloc[0])
    _values.append(v[-1].iloc[0][0])
    values.append(_values)
important_keys.extend(["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"])
df = pd.DataFrame(values, columns=important_keys)
df.sort_values(by=['class_num'], ascending=True)

,seed,h_dim,class_num,control,beta,gemma,cls_dropout,enc_dropout,model_arch,true_cfc,interchange_layer,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,42,192,5,fewshots-augment-large-epochs,1.0,3.0,0.1,0.1,bert-base-uncased,5,12,0.6071,0.4414,0.4658,0.695161
1,42,192,5,fewshots-augment-large-epochs,1.0,3.0,0.1,0.1,bert-base-uncased,50,12,0.5842,0.4192,0.4370,0.700121
2,42,192,5,fewshots-augment-large-epochs,1.0,3.0,0.1,0.1,bert-base-uncased,200,12,0.5444,0.4093,0.3890,0.702815
3,42,192,5,fewshots-augment-large-epochs,1.0,3.0,0.1,0.1,bert-base-uncased,600,12,0.5066,0.3913,0.3368,0.707224
4,42,192,5,fewshots-augment-large-epochs,1.0,3.0,0.1,0.1,bert-base-uncased,1200,12,0.5001,0.3850,0.3169,0.708701
5,42,192,5,fewshots-augment-large-epochs,1.0,3.0,0.1,0.1,bert-base-uncased,1755,12,0.4921,0.3903,0.3070,0.702437


#### Save your results somewhere and load again to tabularize your results altogether

In [28]:
output_name = input("Plase give an output file name: ")

output_directory = f'../proxy_training_results/{model_path}/'
output_filename = os.path.join(output_directory, f'{output_name}.pkl')
print("Writing to file: ", output_filename)
with open(output_filename, 'wb') as f:
    pickle.dump(results, f)

Plase give an output file name: results
Writing to file:  ../proxy_training_results/BERT-fewshots-hdim-results/results.pkl
